<a href="https://colab.research.google.com/github/Parsa33033/ADM_HW3/blob/main/homework3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ADM Homework 3

In [ ]:
from google.colab import drive, files
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


In [107]:
import requests
import os
from bs4 import BeautifulSoup
from tqdm import tqdm
from urllib.request import Request, urlopen
import pandas as pd
import csv
import re
import time
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import pickle
import numpy as np
import collections
import heapq as hq

## 1. Data collection

In [108]:
num_of_pages = 384
directory = ""
links_file = "anime.csv"

### 1.1. Get the list of animes

In [ ]:
anime = []

                               
if not os.path.isfile(links_file):
    with open(directory + links_file, "w", encoding='utf-8', newline='') as fobj:
        fobj.write("page,name,url\n")
        for page in tqdm(range(0, num_of_pages)):
            url = 'https://myanimelist.net/topanime.php?limit=' + str(page * 50)
            response = requests.get(url)

            soup = BeautifulSoup(response.text, 'html.parser')
            for tag in soup.find_all('tr'):
                links = tag.find_all('a')
                for link in links:        
                    if type(link.get('id')) == str and len(link.contents[0]) > 1:
                        x = (page + 1, link.contents[0], link.get('href'))
                        anime.append(x)
                        writer = csv.writer(fobj)
                        writer.writerow(x)

100%|██████████| 384/384 [02:53<00:00,  2.21it/s]


### 1.2. Crawl animes

In [4]:
pages_dir = directory + "pages"
page_dir = pages_dir + "/" + "page"
article = "article"

In [ ]:
if not os.path.exists(pages_dir):
    os.makedirs(pages_dir)
    
def urlEncodeNonAscii(b):
    return re.sub('[\x80-\xFF]', lambda c: '%%%02x' % ord(c.group(0)), b)

ds = pd.read_csv(directory + links_file)
i = 1
for p, u in tqdm(zip(ds.page, ds.url)):
    d = page_dir + str(p)
    if not os.path.exists(d):
        os.makedirs(d)
    d = page_dir + str(p) + "/" + article + "_" + str(i) + ".html"
    
    if not os.path.exists(d):
#         request = requests.get(u, headers={'Cache-Control': 'no-cache', 'User-Agent': 'Mozilla/5.0'})
        req = Request(u, headers={'User-Agent': 'XYZ/3.0'})
        webpage = urlopen(req, timeout=100).read()
        with open(d, "w", encoding='utf-8') as fobj:
            fobj.write(webpage)
    i += 1

### 1.3 Parse downloaded pages

In [5]:
def getText(parent):
    return ''.join(parent.find_all(text=True, recursive=False)).strip()

In [86]:
def article_parser(article_path, soup):
        anime_title = soup.title.text.strip()
        anime_type = ""
        try:
            anime_type = getText(soup.find('span', text="Type:").parent.a)
        except:
            pass
        anime_num_episodes = ""
        try:
            anime_num_episodes = getText(soup.find('span', text="Episodes:").parent)
        except:
            pass
        anime_aired = ""
        try:
            anime_aired = getText(soup.find('span', text="Aired:").parent).split(" to ")
        except:
            pass
        anime_status = ""
        try:
            anime_status = getText(soup.find('span', text="Status:").parent)
        except:
            pass
        anime_score = ""
        try:
            anime_score = getText(soup.find('span', text="Score:").parent.find_all('span', {'class': 'score-label'})[0])
        except:
            pass
        anime_users = ""
        try:
            anime_users = getText(soup.find('span', text="Members:").parent)
        except:
            pass
        anime_rank = ""
        try:
            anime_rank = getText(soup.find('span', text="Ranked:").parent)
        except:
            pass
        anime_popularity = ""
        try:
            anime_popularity = getText(soup.find('span', text="Popularity:").parent)
        except:
            pass
        anime_description = ""
        try:
            anime_description = getText(soup.find('h2', text="Synopsis").parent.parent.p)
        except:
            pass
        anime_related = []
        try: 
            anime_related = list(set(map(getText,soup.find('h2', text="Related Anime").parent.parent.tr.find_all(lambda tag:tag.name == "a" and tag.href != "")))) 
        except: 
            pass
        anime_characters = []
        try:
            anime_characters = list(filter(None,list(map(lambda tr: re.split('\n+', tr.find_all('td')[1].text.strip())[0], soup.find('h2', text="Characters & Voice Actors").find_next('div').find_all('tr')))))
            anime_characters = [e.split(", ") for e in anime_characters]
        except:
            pass
        anime_voices = []
        try:
            anime_voices = list(filter(None,list(map(lambda tr: re.split('\n+', tr.find_all('td')[0].text.strip())[0], soup.find('h2', text="Characters & Voice Actors").find_next('div').find_all('tr')))))
            anime_voices = [e.split(", ") for e in anime_voices]
        except:
            pass
        anime_staff = []
        try:
            anime_staff = list(map(lambda tr: [re.split("\n+", tr.text.strip())[0].split(', '), re.split("\n+", tr.text.strip())[1]], soup.find('h2', text="Staff").find_next('div').find_all('tr')))
        except:
            pass
        l = [anime_title,
                anime_type,
                anime_num_episodes,
                anime_aired,
                anime_status,
                anime_score,
                anime_users,
                anime_rank,
                anime_popularity,
                anime_description,
                anime_related,
                anime_characters,
                anime_voices,
                anime_staff]
        return l
    

In [ ]:
num_of_docs = 19125
i = 1
with open('anime.csv', 'rb') as urls:
    urls_ds = pd.read_csv(urls)
    with open('anime.tsv', 'w', encoding="utf-8", newline='') as anime:
        for p in range(1, num_of_pages + 1):
            flag = True
            if (flag): 
                article_path = page_dir + str(p) + "/" + article + "_" + str(i) + ".html"
                while os.path.isfile(article_path):
                    article_path = page_dir + str(p) + "/" + article + "_" + str(i) + ".html"
                    if os.path.isfile(article_path):
                        if i % 100 == 0:
                            print(p, "--->", i, "--->", article_path)
                        with open(article_path, 'rb') as html:
                            soup = BeautifulSoup(html, "html.parser")
                            l = article_parser(article_path, soup)
                            l.append(urls_ds.url[i - 1])
                            l.append(urls_ds.name[i - 1])
                            writer = csv.writer(anime, delimiter='\t')
                            writer.writerow(l)
                    i += 1


## 2. Search engine

In [109]:
ds = pd.read_csv("anime.tsv", delimiter='\t')
ds.columns = ['anime_title',
                'anime_type',
                'anime_num_episodes',
                'anime_aired',
                'anime_status',
                'anime_score',
                'anime_users',
                'anime_rank',
                'anime_popularity',
                'anime_description',
                'anime_related',
                'anime_characters',
                'anime_voices',
                'anime_staff',
                'url',
                'title']
len(ds)

18741

In [95]:
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [110]:
ps = PorterStemmer()
stopwords = stopwords.words()
def isNaN(string):
    return string != string
def preprocess(sentence):
    if sentence == "" or isNaN(sentence):
        return []
    text_tokens = nltk.word_tokenize(sentence)
    tokens_without_sw = [ps.stem(word) for word in text_tokens if not word in stopwords and word.isalnum()]
    return tokens_without_sw

### 2.1 Conjunctive query

#### 2.1.1

In [111]:
index_dict = {}
for idx, desc in zip(ds.index, ds.anime_description):
    if idx % 1000 == 0: 
        print("---->", idx)
    preprocessed_list = preprocess(desc)
    for term in preprocessed_list:
        if term in index_dict:
            index_dict[term].append(idx)
        else:
            index_dict[term] = []
a_file = open("preprocessed_index.pkl", "wb")
pickle.dump(index_dict, a_file)


----> 0
----> 1000
----> 2000
----> 3000
----> 4000
----> 5000
----> 6000
----> 7000
----> 8000
----> 9000
----> 10000
----> 11000
----> 12000
----> 13000
----> 14000
----> 15000
----> 16000
----> 17000
----> 18000


In [112]:
a_file = open("preprocessed_index.pkl", "rb")
inverseindex_dict = pickle.load(a_file)

#### 2.1.2

In [115]:
query = input()
query = preprocess(query)
s = set(map(int, np.linspace(0, 19000, 19001)))
for q in query:
    s = s.intersection(inverseindex_dict[q])
ds.iloc[list(s), [-1, 9, -2]]

dragon


,title,anime_description,url
9728,Chaos Dragon: Sekiryuu Seneki,"In 3015, the year of Huanli, two countries, Do...",https://myanimelist.net/anime/30091/Chaos_Drag...
8194,Bada-ui Jeonseol Jangbogo,"In a not too distant future, the government se...",https://myanimelist.net/anime/10194/Bada-ui_Je...
5635,X/1999,"At the millennial edge, the concluding battle ...",https://myanimelist.net/anime/155/X_1999
16387,Kiki to Lala no Mamatte Suteki!,Lala finds a myserious egg that hatches into a...,https://myanimelist.net/anime/29249/Kiki_to_La...
1030,Hayate no Gotoku!!,Dr. Saotome returns from the dead after being ...,https://myanimelist.net/anime/4192/Hayate_no_G...
...,...,...,...
1528,Fairy Tail Movie 2: Dragon Cry,Dragon Cry is a magical artifact of deadly pow...,https://myanimelist.net/anime/30778/Fairy_Tail...
1526,Dragon Ball Kai: Mirai ni Heiwa wo! Goku no Ta...,Special episode about Trunks going back to the...,https://myanimelist.net/anime/11359/Dragon_Bal...
7674,Choegang Top Plate,The storyline is unknown but judging from the ...,https://myanimelist.net/anime/31211/Choegang_T...
17401,Harem Time The Animation,A special governmental program has been introd...,https://myanimelist.net/anime/14991/Harem_Time...


### 2.2 Conjunctive query & Ranking score

#### 2.2.1 

In [117]:
def tf(term, desc_preprocessed):
    return desc_preprocessed.count(term)/ len(desc_preprocessed)
def idf(term, inverseindex_dict):
    return np.log(len(inverseindex_dict)/len(inverseindex_dict[term]))
def tf_idf(term, desc_preprocessed, inverseindex_dict):
    x = tf(term, desc_preprocessed) * idf(term, inverseindex_dict)
    return x

In [ ]:
tfidf_invertedindex_dict = {}
for idx, desc in zip(ds.index, ds.anime_description):
    if idx % 1000 == 0: 
        print("---->", idx)
    preprocessed_list = preprocess(desc)
    for term in preprocessed_list:
        if term in tfidf_invertedindex_dict:
            tfidf = tf_idf(term, preprocessed_list, inverseindex_dict)
            tfidf_invertedindex_dict[term].append((idx, tfidf))
        else:
            tfidf_invertedindex_dict[term] = []
a_file = open("tfidf_invertindex.pkl", "wb")
pickle.dump(tfidf_invertedindex_dict, a_file)

In [211]:
a_file = open("tfidf_invertindex.pkl", "rb")
tfidf_invertedindex_dict = pickle.load(a_file)

In [ ]:
desc_dict = {}
for idx, desc in zip(ds.index, ds.anime_description):
    if idx % 1000 == 0: print(idx)
    desc = preprocess(desc)
    desc_vec = {}
    for d in desc:
        tfidf = 0
        for i in tfidf_invertedindex_dict[d]:
            if idx == i[0]:
                tfidf = i[1]
                break
        desc_vec[d] = tfidf 
    desc_dict[idx] = desc_vec
desc_dict_file = open("desc_dict.pkl", "wb")
pickle.dump(desc_dict, desc_dict_file)

In [212]:
desc_dict_file = open("desc_dict.pkl", "rb")
desc_dict = pickle.load(desc_dict_file)

In [213]:
def cosine_similarity_dict(dict1, dict2):
    s = set(dict1.keys()).intersection(set(dict2.keys()))
    c = 0
    norm1 = 0
    norm2 = 0
    for i in s:
        c += dict1[i] * dict2[i]
    for i in dict1.values():
        norm1 += np.power(i, 2)
    norm1 = np.sqrt(norm1)
    
    for i in dict2.values():
        norm2 += np.power(i, 2)
    norm2 = np.sqrt(norm2)
    
    return c / (norm1 * norm2)

#### 2.2.2

In [218]:
query = input()
query = preprocess(query)
query_vec = {}
for q in query:
    query_vec[q] = tf_idf(q, query, inverseindex_dict)
sim_dict = {}
for idx, desc_tfidf in desc_dict.items():
    sim_dict[idx] = cosine_similarity_dict(query_vec, desc_tfidf)
sim_ds = pd.DataFrame({'similarity': sim_dict.values()})
ds['similarity'] = sim_dict.values()
k = 4
clean_dict = {k: sim_dict[k] for k in sim_dict.keys() if sim_dict[k] != 0.0 and not np.isnan(sim_dict[k])}
clean_dict = {k: v for k, v in sorted(clean_dict.items(), key=lambda item: item[1])}

l = [i for i in clean_dict.keys()]
l = l[-k:]

heap = hq.heapify(l)
ds.iloc[l, [-1, 9, 14, 15]].sort_values('similarity', ascending=False)

saiyan race


<ipython-input-213-c07044aa0f89>:16: RuntimeWarning: invalid value encountered in double_scalars
  return c / (norm1 * norm2)


,similarity,anime_description,url,title
1972,0.443457,"Following the defeat of a great adversary, Gok...",https://myanimelist.net/anime/14837/Dragon_Bal...,Dragon Ball Z Movie 14: Kami to Kami
1437,0.339036,"Bardock, Son Goku's father, is a low-ranking S...",https://myanimelist.net/anime/986/Dragon_Ball_...,Dragon Ball Z Special 1: Tatta Hitori no Saish...
1397,0.263309,There are many street races in the Initial D S...,https://myanimelist.net/anime/452/InuYasha_Mov...,InuYasha Movie 1: Toki wo Koeru Omoi
2248,0.255030,As Goku investigates the destruction of the So...,https://myanimelist.net/anime/901/Dragon_Ball_...,"Dragon Ball Z Movie 08: Moetsukiro!! Nessen, R..."


### A WAY BETTER SOLUTION (way faster and simpler)

In [166]:
query = input()
query = preprocess(query)
s = set(map(int, np.linspace(0, 19000, 19001)))
for q in query:
    l = [x[0] for x in tfidf_invertedindex_dict[q]]
    s = s.intersection(l)
# with open('anime.tsv', 'rb') as anime:
    

dragon


In [167]:
q = [x for x in tfidf_invertedindex_dict[q] if x[0] in s]
q = [x[0] for x in sorted(q, key=lambda x: x[1])]
k = int(input())
ds.iloc[q[:k]]

4


,anime_title,anime_type,anime_num_episodes,anime_aired,anime_status,anime_score,anime_users,anime_rank,anime_popularity,anime_description,anime_related,anime_characters,anime_voices,anime_staff,url,title,similarity
2492,Souten Kouro (Beyond the Heavens) - MyAnimeLis...,TV,26,"['Apr 8, 2009', 'Sep 30, 2009']",Finished Airing,7.27,"16,783",#2546,#4326,Souten Kouro's story is based loosely on the e...,['Souten Kouro'],"[['Cao', 'Cao'], ['Sun', 'Quan'], ['Liu', 'Bei...","[['Miyano', 'Mamoru'], ['Seki', 'Tomokazu'], [...","[[['Tominaga', 'Tsuneo'], 'Director'], [['Hond...",https://myanimelist.net/anime/5226/Souten_Kouro,Souten Kouro,0.043387
2662,Miracle Giants Doumu-kun - MyAnimeList.net,TV,49,"['Apr 2, 1989', 'Mar 25, 1990']",Finished Airing,7.23,"1,416",#2720,#10289,"Dome Shinjo, a 10-year-old boy who is son of a...",['Miracle Giants Doumu-kun Special'],"[['Shinjou', 'Akane'], ['Shinjou', 'Mayumi']]","[['Yukie', 'Rena'], ['Inoue', 'Kikuko']]","[[['Kimura', 'Kyotaro'], 'Producer'], [['Watan...",https://myanimelist.net/anime/5273/Miracle_Gia...,Miracle Giants Doumu-kun,0.056903
3660,"Slime Taoshite 300-nen, Shiranai Uchi ni Level...",TV,12,"['Apr 10, 2021', 'Jun 26, 2021']",Finished Airing,7.02,"247,041",#3738,#732,"In role-playing games, slimes are usually the ...","['Slime Taoshite 300-nen, Shiranai Uchi ni Lev...","[['Azusa'], ['Laika'], ['Falfa'], ['Halkara'],...","[['Yuuki', 'Aoi'], ['Hondo', 'Kaede'], ['Senbo...","[[['Kimura', 'Nobukage'], 'Director'], [['Moto...",https://myanimelist.net/anime/40586/Slime_Taos...,"Slime Taoshite 300-nen, Shiranai Uchi ni Level...",0.068078
406,Tensei shitara Slime Datta Ken - MyAnimeList.net,TV,24,"['Oct 2, 2018', 'Mar 19, 2019']",Finished Airing,8.11,"1,030,225",#416,#100,Thirty-seven-year-old Satoru Mikami is a typic...,['Tensei shitara Slime Datta Ken: Kanwa - Veld...,"[['Tempest', 'Rimuru'], ['Nava', 'Milim'], ['D...","[['Okasaki', 'Miho'], ['Hidaka', 'Rina'], ['Sa...","[[['Kikuchi', 'Yasuhito'], 'Director, Animatio...",https://myanimelist.net/anime/37430/Tensei_shi...,Tensei shitara Slime Datta Ken,0.068884


## 3. Define new score

We decided to define a new score that builds and improves upon the previously used cosine similarity: we assign different weights to a series of parameters, namely the cosine similarity, the anime's type (TV, Movie, Music, etc.), the anime's score, the anime's characters and the staff that worked on it. 
The formula we used looks like this:

Custom score =  cos.similarity * 0.4 + type * 0.3 + characters * 0.05 + staff * 0.05


Type, characters and staff variables are simply calculated as 1 if any word in the query input matches a word in the respective column, and 0 if no words from the query input are found. We felt that this would be a good way to let users filter for what they are interested in in their search: if I want to find a movie from the Dragon Ball serie, I just have to type "Dragon Ball Movie" and the animes classified as movies will be prioritized.
For the "score" variable we simply consider the absolute value of the anime's score (divided by 10 for normalizing purposes), since we wanted to give priority to animes with a higher score.

In [ ]:
inverted_idx_desc = {}
inverted_idx_type = {}
inverted_idx_chars = {}
inverted_idx_staff = {}
for idx, t, chars, staff, desc in zip(ds.index, ds.anime_type, ds.anime_characters, ds.anime_staff, ds.anime_description):
    if idx % 1000 == 0: 
        print("---->", idx)
    preprocessed_list = preprocess(desc)
    for term in preprocessed_list:
        if term in inverted_idx_desc:
            tfidf = tf_idf(term, preprocessed_list, inverseindex_dict)
            inverted_idx_desc[term].append((idx, tfidf))
        else:
            inverted_idx_desc[term] = []
    
    if t in inverted_idx_type:
            inverted_idx_type[t].append(idx)
    else:
        inverted_idx_type[t] = []
            
    for char_name in chars:
        if char_name in inverted_idx_chars:
            inverted_idx_chars[char_name].append(idx)
        else:
            inverted_idx_chars[char_name] = []
        
    for staff_name in staff:
        if staff_name in inverted_idx_staff:
            inverted_idx_staff[staff_name].append(idx)
        else:
            inverted_idx_staff[staff_name] = []   
    
desc_file = open("invertindex_desc.pkl", "wb")
pickle.dump(inverted_idx_desc, desc_file)

type_file = open("invertindex_type.pkl", "wb")
pickle.dump(inverted_idx_type, type_file)

chars_file = open("invertindex_chars.pkl", "wb")
pickle.dump(inverted_idx_chars, chars_file)

staff_file = open("invertindex_staff.pkl", "wb")
pickle.dump(inverted_idx_staff, staff_file)

In [177]:
desc_file = open("invertindex_desc.pkl", "rb")
inverted_idx_desc = pickle.load(desc_file)

type_file = open("invertindex_type.pkl", "rb")
inverted_idx_type = pickle.load(type_file)

chars_file = open("invertindex_chars.pkl", "rb")
inverted_idx_chars = pickle.load(chars_file)

staff_file = open("invertindex_staff.pkl", "rb")
inverted_idx_staff = pickle.load(staff_file)

In [180]:
desc_dict_file = open("desc_dict.pkl", "rb")
desc_dict = pickle.load(desc_dict_file)

In [ ]:

for idx, t, chars, staff, desc in zip(ds.index, ds.anime_type, ds.anime_characters, ds.anime_staff, ds.anime_description):
    if idx % 1000 == 0: print(idx)
    desc = preprocess(desc)
    desc_vec = {}
    for d in desc:
        tfidf = 0
        for i in tfidf_invertedindex_dict[d]:
            if idx == i[0]:
                tfidf = i[1]
                break
        desc_vec[d] = tfidf 
    desc_dict[idx] = {'desc': desc_vec, 'chars': chars, 'type': t, 'staff': staff}
big_dict_file = open("big_dict.pkl", "wb")
pickle.dump(desc_dict, big_dict_file)

In [183]:
big_dict_file = open("big_dict.pkl", "rb")
big_dict = pickle.load(big_dict_file)

In [220]:
query = input()
query = preprocess(query)
query_vec = {}
for q in query:
    query_vec[q] = tf_idf(q, query, inverseindex_dict)
sim_dict = {}
for idx, d in big_dict.items():
    t, chars, staff, desc = [d['type'], d['chars'], d['staff'], d['desc']]
    t = str(t)
    t = t.lower()

    t_num = 0
    if t in q:
        t_num = 1
    c_num = 0
    for c in chars:
        for q in query:
            if c == q:
                c_num += 1
    s_num = 0
    for s in chars:
        for q in query:
            if s == q:
                s_num += 1
    
    sim_dict[idx] = cosine_similarity_dict(query_vec, desc) * 0.25 +  t_num * 0.25 + c_num * 0.25 + s_num * 0.25

sim_ds = pd.DataFrame({'similarity': sim_dict.values()})
ds['similarity'] = sim_dict.values()
k = 4
clean_dict = {k: sim_dict[k] for k in sim_dict.keys() if sim_dict[k] != 0.0 and not np.isnan(sim_dict[k])}
clean_dict = {k: v for k, v in sorted(clean_dict.items(), key=lambda item: item[1])}

l = [i for i in clean_dict.keys()]
l = l[-k:]

heap = hq.heapify(l)
ds.iloc[l, [-1, 9, 14, 15]].sort_values('similarity', ascending=False)

saiyan race movie goku


<ipython-input-213-c07044aa0f89>:16: RuntimeWarning: invalid value encountered in double_scalars
  return c / (norm1 * norm2)


,similarity,anime_description,url,title
1437,0.099594,"Bardock, Son Goku's father, is a low-ranking S...",https://myanimelist.net/anime/986/Dragon_Ball_...,Dragon Ball Z Special 1: Tatta Hitori no Saish...
5547,0.093649,"One peaceful afternoon, the Son family and fri...",https://myanimelist.net/anime/22695/Dragon_Bal...,Dragon Ball Z: Summer Vacation Special
4890,0.090904,Specials included with the original series.\n\...,https://myanimelist.net/anime/2520/Dragon_Ball...,Dragon Ball Specials
3158,0.089819,A retelling of Dragon Ball's origin with a dif...,https://myanimelist.net/anime/2748/Dash_Kappei,Dash! Kappei


## 4. BONUS: Understanding the anime's reviews

In [294]:
def article_parser(article_path, soup):
        anime_title = soup.title.text.strip()
        try:
            anime_review = [x.strip() for x in re.split('(<br>|\n+)', soup.find('a', text="More reviews").find_next('div').text) if len(x) > 100]
        except:
            pass
        l = [anime_title, anime_review[:5]]
        return l
    

In [296]:
num_of_docs = 19125
i = 1
with open('anime.csv', 'rb') as urls:
    urls_ds = pd.read_csv(urls)
    with open('anime_review.tsv', 'w', encoding="utf-8", newline='') as anime:
        for p in range(1, num_of_pages + 1):
            flag = True
            if (flag): 
                article_path = page_dir + str(p) + "/" + article + "_" + str(i) + ".html"
                while os.path.isfile(article_path):
                    article_path = page_dir + str(p) + "/" + article + "_" + str(i) + ".html"
                    if os.path.isfile(article_path):
                        if i % 100 == 0:
                            print(p, "--->", i, "--->", article_path)
                        with open(article_path, 'rb') as html:
                            soup = BeautifulSoup(html, "html.parser")
                            l = article_parser(article_path, soup)
                            l.append(urls_ds.url[i - 1])
                            l.append(urls_ds.name[i - 1])
                            writer = csv.writer(anime, delimiter='\t')
                            writer.writerow(l)
                    i += 1


2 ---> 100 ---> pages/page2/article_100.html
4 ---> 200 ---> pages/page4/article_200.html
6 ---> 300 ---> pages/page6/article_300.html


KeyboardInterrupt: 

In [ ]:
"".join(review.split(".")[:3])

## 5. Algorithmic question

 In this problem, the goal is to maximize the sum of values contained in a list, where each value represents the time that the person who made the appointment wants to spend with the personal trainer, so you want to provide the personal trainer with a program that maximizes the total length of accepted appointments. 
The important condition is that the personal trainer needs a break between appointments and cannot accept two consecutive bookings. In addition, reservations are considered in the order in which they were made, so that the personal trainer cannot go back and accept past bookings, but must follow the chronological order. 

Therefore, considering the need for a break between one appointment and another and considering that past appointments cannot be accepted but it is necessary to keep to the chronological order, one solution could be to scroll through the list of bookings twice, that is once starting from the first reservation and then from the index 0, or even, and continuing with step 2 for all even indices and scroll the list a second time starting from the second reservation and then from the index 1, or odd, and proceeding with step 2 for all odd indices. 
Then add the values of the even and odd indices and compare the obtained values. The path that maximizes the sum, will be preferred. 


In [28]:
def appointments(appointments):
    l1 = []
    res1 = 0
    l2 = []
    res2 = 0
    for i in range(len(appointments)):
        if ((i % 2) == 0):
            res1 += appointments[i]
            l1.append(appointments[i])
        else:
            res2 += appointments[i]
            l2.append(appointments[i])
    if (res1 > res2):
        return l1, res1
    else:
        return l2, res2

In [29]:
mylist = [30, 40, 25, 50, 30, 20] 
print('The list is:', mylist, '\n') 
 
print('The best way to accept reservations in the given order is:', appointments(mylist)[0], 'with max duration of:', appointments(mylist)[1])

The list is: [30, 40, 25, 50, 30, 20] 

The best way to accept reservations in the given order is: [40, 50, 20] with max duration of: 110
